In [60]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [61]:
def normalized(a, axis=-1, order=2):
    l2 = np.atleast_1d(np.linalg.norm(a, order, axis))
    l2[l2==0] = 1
    return a / np.expand_dims(l2, axis)

In [62]:
def flatten_image_vectors(image):
    """
    Takes in the image as np array of size 256x256 pixels and returns an np array of sixe 1 x (256x256)
    """
    #Resize each image to 256 x 256 (sanity check)
#     image = cv2.resize(image, (256,256), interpolation = cv2.INTER_AREA) 
    return image.flatten()

In [63]:
def stack_vectors(images):
    stack = []
    for img in images:
        #Size of the image is 65536 x 1
        img = flatten_image_vectors(img)
        #Convert size of the image to 65536 x 1
        img = normalized(img,0).T

        stack.append(np.array(img))
    #Size of the stack is (563, 65536)
    stack = np.squeeze(np.array(stack),axis=2)
    return stack


In [64]:
def compute_mean_vector(stack):
    mean_vector = []
    for var in range(stack.shape[0]):
        mean = np.mean(stack[var, :])
        mean_vector.append(mean)
    
    return np.array(mean_vector)

def compute_mean_image(stack):
    return compute_mean_vector(stack.T)

In [65]:
def compute_scatter_matrix(stack, mean_vector):
    scatter_matrix = np.zeros((mean_vector.shape[0],mean_vector.shape[0]))
    for var in range(stack.shape[1]):
        scatter_matrix += (stack[:,var].reshape(mean_vector.shape[0],1) - mean_vector).dot((stack[:,var].reshape(mean_vector.shape[0],1) - mean_vector).T)

    return scatter_matrix

In [66]:
def compute_covariance_matrix(stack):
    cov_matrix = []
    for var in range(stack.shape[0]):
        cov_matrix.append(stack[var, :])

    return np.cov(cov_matrix)

In [67]:
def get_eigenvecs_vals(matrix):
    eigenvals, eigenvecs = np.linalg.eig(matrix)
    return eigenvals, eigenvecs


In [68]:
def sort_eigenvec_by_eigenval(eigenvals, eigenvecs):
    """
    1. Make a list of (eigenvalue, eigenvector) tuples
    2. Sort the (eigenvalue, eigenvector) tuples from high to low
    """
    eig_pairs = [(np.abs(eigenvals[i]), eigenvecs[:,i]) for i in range(len(eigenvals))]
    eig_pairs.sort(key=lambda x: x[0], reverse=True)

    return  eig_pairs


In [69]:
def compute_covariance_matrix(stack):
    cov_matrix = []
    for var in range(stack.shape[0]):
        cov_matrix.append(stack[var, :])

    return np.cov(np.array(cov_matrix))

In [70]:
def choose_k_eigenvecs(eig_pairs, k, image_stack):
    stack_tuple = (eig_pairs[0][1].reshape(eig_pairs[0][1].shape[0],1),)
    for var in range(1,k):
        stack_tuple += (eig_pairs[var][1].reshape(eig_pairs[var][1].shape[0],1),)

    matrix_w = np.hstack(stack_tuple)
    print("eig_pairs[var][1].shape[0] : ",eig_pairs[var][1].shape[0])
    for var in range(image_stack.shape[1]-eig_pairs[var][1].shape[0]):
        matrix_w = np.append(matrix_w, np.array([np.zeros(k)]), axis=0)
    print(matrix_w)
    
    print("Shape of the matrix W is {}".format(matrix_w.shape))
    return matrix_w


In [71]:
def get_transformed_images(matrix_w,stack):
    """
    The dimension of the transformed matrix k x number_of_images  
    """
    transformed = matrix_w.T.dot(stack)
    print("Shape of the transformed matrix is {}".format(transformed.shape))
    return transformed.T

In [72]:
def find_closest_image(transformed, matrix_w, image, mean_vector):
    img = flatten_image_vectors(image)
    img = normalized(img,0).T
    img = np.squeeze(img, axis=1)
    print("image: ",img)
    print("mean_image: ",mean_vector)
#     img = img - mean_vector
    print("image after subtracting mean: ",img)
    y_hat = matrix_w.T.dot(img)
#     reconstruct_image(matrix_w, y_hat, mean_image)
    print(y_hat)
    print("transformed[0]: ", transformed[0])
    print("y_hat - transformed[0]: ", y_hat-transformed[0])
    
    argmin = np.linalg.norm(y_hat-transformed[0])
    image_id = 0
    
    top_10_max_vals = [argmin for _ in range(10)]
    top_10_closest_img = [image_id for _ in range(10)]
    for var in range(1, transformed.shape[0]):
        dist = np.linalg.norm(y_hat-transformed[var])
        for arg in range(1,len(top_10_max_vals)):
            if dist < top_10_max_vals[arg]:
                top_10_max_vals[arg-1] = top_10_max_vals[arg]
                top_10_max_vals[arg] = dist
                top_10_closest_img[arg-1] = top_10_closest_img[arg]
                top_10_closest_img[arg] = var
                break
    print("max vals: ",top_10_max_vals)

    return image_id

In [73]:
def sort_eigenvec_by_eigenval(eigenvals, eigenvecs):
    """
    1. Make a list of (eigenvalue, eigenvector) tuples
    2. Sort the (eigenvalue, eigenvector) tuples from high to low
    """
    eig_pairs = [(np.abs(eigenvals[i]), eigenvecs[:,i]) for i in range(len(eigenvals))]
    eig_pairs.sort(key=lambda x: x[0], reverse=True)

    return  eig_pairs

In [74]:
def reconstruct_image(matrix_w, eigencoeff, mean_image):
    image = matrix_w.dot(eigencoeff) 
    image = (image * 256).astype(int)
    image = image.reshape(2,2)
    cv2.imwrite("image.jpg",image)
    print("reconstructed: ",image)

In [75]:
IMG_THRESHOLD = 15
k = 2

In [76]:
images = np.array([[1, 2, 3, 4], [4, 5, 6, 7], [7, 8, 9, 1]])

In [77]:
image_stack = stack_vectors(images)
print(image_stack.shape)
mean_vector = compute_mean_vector(image_stack)
mean_image = compute_mean_image(image_stack)
scatter_matrix = compute_scatter_matrix(image_stack,mean_vector)
print(scatter_matrix.shape)
cov_matrix = compute_covariance_matrix(image_stack)
print(cov_matrix.shape)

(3, 4)
(3, 3)
(3, 3)


In [78]:
# eigenvectors and eigenvalues for the from the scatter matrix
eig_val_sc, eig_vec_sc = get_eigenvecs_vals(scatter_matrix)

# eigenvectors and eigenvalues for the from the covariance matrix
eig_val_cov, eig_vec_cov = get_eigenvecs_vals(cov_matrix)

eig_pairs1 = sort_eigenvec_by_eigenval(eig_val_sc, eig_vec_sc)
eig_pairs2 = sort_eigenvec_by_eigenval(eig_val_cov, eig_vec_cov)

matrix_w_1 = choose_k_eigenvecs(eig_pairs1, k, image_stack)
matrix_w_2 = choose_k_eigenvecs(eig_pairs2, k, image_stack)

transformed1 = get_transformed_images(matrix_w_1, image_stack.T)
transformed2 = get_transformed_images(matrix_w_2, image_stack.T)

eig_pairs[var][1].shape[0] :  3
[[-0.64295127 -0.62562014]
 [-0.31955425 -0.30514273]
 [ 0.69605944 -0.71797449]
 [ 0.          0.        ]]
Shape of the matrix W is (4, 2)
eig_pairs[var][1].shape[0] :  3
[[-0.64521434 -0.62561231]
 [-0.31483236 -0.30526755]
 [ 0.69611712 -0.71792824]
 [ 0.          0.        ]]
Shape of the matrix W is (4, 2)
Shape of the transformed matrix is (2, 3)
Shape of the transformed matrix is (2, 3)


In [79]:
img_id1 = find_closest_image(transformed1, matrix_w_1, np.array([7, 8, 8, 1]), mean_image)

image:  [0.52467218 0.59962535 0.59962535 0.07495317]
mean_image:  [0.34673431 0.46115856 0.57558281 0.4751726 ]
image after subtracting mean:  [0.52467218 0.59962535 0.59962535 0.07495317]
[-0.11157659 -0.9417325 ]
transformed[0]:  [ 0.14717644 -0.61889528]
y_hat - transformed[0]:  [-0.25875303 -0.32283722]
max vals:  [0.22852763329520334, 0.055625380826017125, 0.4137354241948184, 0.4137354241948184, 0.4137354241948184, 0.4137354241948184, 0.4137354241948184, 0.4137354241948184, 0.4137354241948184, 0.4137354241948184]


In [80]:
print(img_id1)

0


In [81]:
#images = np.array([[1, 2, 3, 4], [4, 5, 6, 7], [7, 8, 9, 1], [4, 6, 6, 7], [7, 8, 7, 1], [8, 5, 6, 7], [7, 8, 1, 1], [4, 5, 6, 10], [2, 8, 9, 1], [1, 5, 6, 3], [2, 8, 2, 1]])